# RAG检索增强

## RAG是什么？

RAG全称Retrieval augmented generation
LLM是在一个固定庞大的数据集上训练的，这就导致一些数据他获取不到，比如最新的数据，虽然LLM提供了微调的方式，但这种方式也比较昂贵，所以就有了RAG，RAG是将相关的信息提供给LLM，基于这些信息来回答问题。所以，他是一种便宜并且实用的技术。

RAG大体步骤如下

1. 输入问题
2. 在资料中找到和这个问题相关的数据
3. 组成Prompt，和LLM交互


## Embedding

解决的是如何通过输入的问题找到相关的数据，这就说到向量化了。
向量化就是将输入的一段文本或者任意形式的数据转换为一组数字，这些数字表示的是这组数据的特点，这些数据在数学上用向量表示，存储这些数据的数据库叫做向量数据库。
具体可看：https://guangzhengli.com/blog/zh/vector-database/

## RAG架构
具体可看：https://python.langchain.com/v0.1/docs/use_cases/question_answering/#rag-architecture
主要有两点
1. Indexing
    做数据索引和向量化
2. 检索和生成
    实际的RAG链，在运行时接收用户查询并从索引中检索相关数据，然后将其传递给模型。

### Indexing
![](../resource/img_6.png)
分为三步
1. 加载
    加载数据，通过 DocumentLoaders.
2. 切分
    上一步加载后返回了Documents对象，这一步将它切分为小块，为了更好的索引和存储，大的快更难搜索，并且无法适应LLM的有限上下文窗口。
3. 存储
    将特征值存储
### 检索和生成
![](../resource/img_7.png)
1. 检索
    从存储中找到相关的数据，使用Retriever.
2. 生成
    在生成的Prompt中嵌入相关数据和用户输入问题，llm生成答案

所以，在这章节中，步骤如下
1. 加载资源
2. 做Embedding，存储到向量数据库
3. 输入问题，Embedding，在向量数据库中找相关文档
4. 组成Prompt，和LLM交互

## 加载资源
LangChain提供了很多的加载器，具体可看 

https://api.python.langchain.com/en/latest/community_api_reference.html#module-langchain_community.document_loaders

https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/

在这里，我们加载html

In [23]:
from langchain_community.document_loaders import WebBaseLoader
import langchain 
langchain.debug=True
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://daliuchen.github.io/langchain-guide/intro.html",)
)
docs = loader.load()
print(docs)

[Document(page_content='\n\n\n\n\n\n欢迎来到我的langchain的学习手册 — langchan study guide\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\nBack to top\n\n\n\n\n\n\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                    欢迎来到我的langchain的学习手册\n                \n\n\n\nhappy path\nLangChain Expression Language\nPrompt templates\nExample selectors\nOutput parsers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRepository\n\n\n\n\n\n\nOpen issue\n\n\n\n\n\n\n\n\n\n\n\n\n\n.md\n\n\n\n\n\n\n\n.pdf\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n欢迎来到我的langchain的学习手册\n\n\n\n\n Contents \n\n\n\n欢迎来到我的langchain的学习手册\nlangchain总述\n一句话说清\n是什么？\n特点\n整体架构\n\n\n\n\n\n\n\n\n\n\n欢迎来到我的langchain的学习手册#\nlangchain最近很火热，在这里记录我对langchain的学习。\n欢迎大家一块添砖加瓦\n\n\nlangchain总述#\nlangchain官网：\nhttps://python.langchain.com/v0.2/docs/introduction/\n\n一句话说清#\n在和LLM交互的时候，首先需要写prompt，在调用LLM提供的API，解析输出。这里面就有三步，langchain将这三步抽象简化，并且提供

## 切分chunk

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200) # 每500个字符为一块，两个块之间交替重合200个字符
splits = text_splitter.split_documents(docs)
print(splits)
print(len(splits))

[Document(page_content='欢迎来到我的langchain的学习手册 — langchan study guide\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\nBack to top\n\n\n\n\n\n\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                    欢迎来到我的langchain的学习手册\n                \n\n\n\nhappy path\nLangChain Expression Language\nPrompt templates\nExample selectors\nOutput parsers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRepository\n\n\n\n\n\n\nOpen issue\n\n\n\n\n\n\n\n\n\n\n\n\n\n.md\n\n\n\n\n\n\n\n.pdf\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n欢迎来到我的langchain的学习手册\n\n\n\n\n Contents', metadata={'source': 'https://daliuchen.github.io/langchain-guide/intro.html', 'title': '欢迎来到我的langchain的学习手册 — langchan study guide', 'language': 'en'}), Document(page_content='Repository\n\n\n\n\n\n\nOpen issue\n\n\n\n\n\n\n\n\n\n\n\n\n\n.md\n\n\n\n\n\n\n\n.pdf\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n欢迎来到我的langchain的学习手册\n\n\n\n\n Co

## Embedding

In [30]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
vectorstore.similarity_search("LangChain特点",k=1)

[Document(page_content='一句话说清#\n在和LLM交互的时候，首先需要写prompt，在调用LLM提供的API，解析输出。这里面就有三步，langchain将这三步抽象简化，并且提供了很多组件，他有强大的社区，\n有很多好用的东西，方便我们开发LLM的应用程序\n\n\n是什么？#\n\n基于大语言模型的开发框架（LLM）\n大语言模型的一站式开发框架\n\n\n\n特点#\n\n简化了大语言模型开发的难度，将和模型交互的各个阶段做抽象组合。\n提供了一站式的开发框架，包括开发，部署，观测\n简化了llm应用生命周期阶段，包括\n\n开发：Langchain提供了很多的组件，模块来构建应用程序，并且有有强大的社区生态。\n生产：LangSmith可以检查、监控和评估chain。\n部署：LangServe可以将chain暴露给外部服务来使用（API）\n\n\n\n\n\n整体架构#\n\n如上图所示，langchain的库如下', metadata={'language': 'en', 'source': 'https://daliuchen.github.io/langchain-guide/intro.html', 'title': '欢迎来到我的langchain的学习手册 — langchan study guide'})]

## 套入Prompt中生成

In [31]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt") # 从LangChainhub上拿到已经写好的promot
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]

In [32]:
# 从上面看到，有两个参数，context，和question
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough


llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )

rag_chain.invoke("LangChain是什么？")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "LangChain是什么？"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "LangChain是什么？"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "LangChain是什么？"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "LangChain是什么？"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question>] [1.85s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering

'LangChain是一个允许用户与LLM交互的平台，简化了写prompt、调用API和解析输出的步骤，并提供了许多组件和社区支持。您可以在https://python.langchain.com/v0.2/docs/introduction/找到官方网站。'

虽然上面已经拿到了答案，但是细看promot中发现，在从向量数据库中查找完后，format Prompt的时候没有format Document。
所以，应该添加一个方法，在检索之后

In [38]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
   
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain.invoke("LangChain的特点是什么")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "LangChain的特点是什么"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "LangChain的特点是什么"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "LangChain的特点是什么"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "LangChain的特点是什么"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "LangChain的特点是什么"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence > chain:format_docs] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<co

'LangChain是基于大语言模型（LLM）的开发框架，简化了大语言模型开发的难度并提供了一站式的开发框架，包括开发、部署和观测。它还简化了LLM应用生命周期阶段，包括提供组件、模块来构建应用程序，监控和评估以及将chain暴露给外部服务使用。LangChain具有强大的社区生态，方便开发LLM的应用程序。'

这一章只是介绍了RAG的基本思路和简单demo，在后续，会有复杂有趣的示例。